<a href="https://colab.research.google.com/github/zhangyuyangtoushui/Surge4Advanced/blob/master/Torrent_To_Google_Drive_Downloader_v4_1_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader v4.1 v2

### Mount Google Drive
To stream files we need to mount Google Drive.

In [1]:
#@markdown <br><center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height="50" alt="Gdrive-logo' "/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


### Extras

**Google Colab Timeout Preventer! Run this Code to prevent TIMEOUT in Google Colab.Before running any code run this!**

In [2]:
#@markdown <br><center><img src='https://github.com/DarkXonline/Torrent-To-GDrive-Download/blob/main/imgs/2.png?raw=true' height="100" alt="netdata"/></center>
#@title <center>Time Out Preventer (Advanced) </center></strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent NOTEBOOK from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button");
      btn.click()
      }

    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect");
      btn.click()
      }
    }

  setInterval(ClickConnect,60000)
  '''))

**Check from which speed Google Will transfer your Files: (Optional)**

In [3]:
#@markdown <br><center><img src='https://github.com/DarkXonline/Torrent-To-GDrive-Download/blob/main/imgs/1.jpg?raw=true' height="50" alt="ST-logo"/></center>
#@markdown <center><h3>Check Transfer Speed!</h3></center><br>
!curl -s https://raw.githubusercontent.com/sivel/speedtest-cli/master/speedtest.py | python

Retrieving speedtest.net configuration...
<stdin>:960: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
Testing from Google Cloud (34.125.209.252)...
Retrieving speedtest.net server list...
Selecting best server based on ping...
Hosted by ALLO - Las Vegas (Las Vegas, NV) [2.03 km]: 18.273 ms
Testing download speed................................................................................
Download: 2518.17 Mbit/s
Testing upload speed......................................................................................................
Upload: 610.94 Mbit/s


###Dependency
https://www.libtorrent.org/

In [4]:
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent && apt install python3-libtorrent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.1 MB/s  0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libboost-python1.74.0 libtorrent-rasterbar2.0
The following NEW package

### Code to download torrent

In [5]:
#@markdown <center><h3>Start Download to Google Drive</h3></center><br>
import libtorrent as lt
import time
import datetime
import sys

# User Input
link = "\"Input Torrent Link or Magnet here\"" #@param {type:"string"}
#@markdown Change zipp to **"Yes"** if you want to zip your file.
zipp = "No" #@param ["No", "Yes"]

#@markdown Custom path inside the /Torrent/ (if needed) [ended with **'/'**] :
Destination_Path = "" #@param {type:"string"}

if link == "\"Input Torrent Link or Magnet here\"" or link == '':
  print("**Please input Torrent Link or Magnet**")
  sys.exit()

def download_torrent():
    ses = lt.session()
    ses.listen_on(6881, 6891)
    # link = input("Input Torrent Link or Magnet and Press Enter: ")
    print(link)

    handle = lt.add_magnet_uri(ses, link, params)
    # change the 0 to a 1 to download sequential - this sequential option is only if you selected zip. If not,
    # scroll farther down.
    handle.set_sequential_download(0)

    ses.start_dht()
    begin = time.time()
    print(datetime.datetime.now())

    print('Downloading Metadata...')
    while not handle.has_metadata():
        time.sleep(1)
    print('Got Metadata, Starting Torrent Download...')

    print("Starting", handle.name())

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata',
                     'downloading', 'finished', 'seeding', 'allocating']
        print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
              (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
               s.num_peers, state_str[s.state]))
        time.sleep(5)

    end = time.time()
    print(handle.name(), "COMPLETE")

    print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")

    print(datetime.datetime.now())

default_path = '/content/drive/MyDrive/Torrent/'

if Destination_Path != '':
  final_path = default_path + Destination_Path
else:
  final_path = default_path

if zipp == 'Yes' or zipp == 'y':
    #@markdown Enter zip name if zipp set to **"Yes"** :
    zip_name = "\"Enter zip name\"" #@param {type:"string"}
    import shutil

    params = {
        'save_path': '/content/temp/',
        'storage_mode': lt.storage_mode_t(2),
    }
    download_torrent()

    print("now time to zip")

    print("Zipping ...")
    final_zip_directory = final_path + zip_name
    shutil.make_archive(final_zip_directory, 'zip', '/content/temp/')
    shutil.rmtree('/content/temp/')

else:
    params = {
        'save_path': final_path,
        'storage_mode': lt.storage_mode_t(2),
    }
    download_torrent()

print('\nYour file saved at', final_path)
print('ALL DONE!')


ImportError: /usr/local/lib/python3.12/dist-packages/libtorrent.so: undefined symbol: PyUnicode_AsUnicode